In [ ]:
import numpy as np
import pandas as pd
from statsmodels.formula.api import ols
import plotnine as gg

from ModelTools.regression.linear_model import LinearModel

In [ ]:
rng = np.random.default_rng(1)
n = 100
data = pd.DataFrame({
    'x1':rng.normal(size=n),
    'x2':rng.normal(size=n).cumsum(),
    'x3':rng.normal(size=n),
    'x4':rng.normal(size=n)
}).assign(
    y1=lambda dt:rng.normal(1+1*dt.x1+2*dt.x2+3*dt.x3+4*dt.x4+0.4*dt.x4**2,scale=1)+rng.standard_t(df=1,size=n),
    y2=lambda dt:rng.normal(1+2*dt.x2,scale=1),
    y3=lambda dt:rng.normal(1+(dt.x2),scale=100*np.exp(dt.x2)/(1+np.exp(dt.x2)))
)



formula = 'y2~x1+x2'
mod=LinearModel(formula,data=data).fit(method='OLS')

mod.get_coef()

In [ ]:
mod.plot_coef_dist()

In [ ]:
mod.get_metric(bootstrap=False)

In [ ]:
mod.plot_prediction(data_grid={'x2':np.linspace(-9,2,30),'x1':[1,10],'x3':[1,2],'x4':[1,2]})

In [ ]:
mod.plot_check()

In [ ]:
np.random.ch

In [ ]:
mod.get_metric(bootstrap=True)

In [ ]:
gg.options.figure_size=[9,9]
(
    mod.get_metric(bootstrap=True,summary=False)
    .melt()
    .pipe(gg.ggplot)
    + gg.aes(x='value')
    + gg.geom_density()
    + gg.facet_wrap('variable',scales='free')
)

In [ ]:
ols(formula,data=data).fit().summary()

In [ ]:
(
    mod.predict(ci_method='bootstrap',alpha=0.05)
    .pipe(gg.ggplot)
    + gg.aes(x='x1')
    # + gg.geom_point(gg.aes(y='y'))
    + gg.geom_line(gg.aes(y='mean'),color='red')
    + gg.geom_ribbon(gg.aes(ymin='obs_ci_lower',ymax='obs_ci_upper'),alpha=0.3)
    + gg.geom_ribbon(gg.aes(ymin='mean_ci_lower',ymax='mean_ci_upper'),alpha=0.3,fill='red')
    # + gg.stat_function(fun=lambda x:np.sin(x),color='blue')
)

In [ ]:
(
    mod.coef_dist_boot
    .melt()
    .pipe(gg.ggplot)
    + gg.aes(x='value')
    + gg.geom_density()
    + gg.facet_wrap('variable',ncol=1,scales='free_y')
)

In [ ]:
(
    mod.coef_dist_boot
    .pipe(gg.ggplot)
    + gg.aes(x='Intercept',y='x1')
    + gg.geom_pointdensity()
    + gg.geom_density_2d()
)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from statsmodels.nonparametric.smoothers_lowess import lowess
from scipy.stats import norm
from scipy.stats import gaussian_kde

plt.style.use('ggplot')
%config InlineBackend.figure_format = 'retina'

def plot_check_model(
    residual,
    fitted_value,
    boot_pred,
    y_name='y'
):
    y_value = fitted_value + residual
    std_residual = (residual - np.mean(residual))/np.std(residual)
    abs_std_residual = np.abs(std_residual)
    fig,ax = plt.subplots(ncols=2,nrows=2,figsize=[10,6])
    fig.tight_layout(h_pad=5,w_pad=2)
    fontdict = {'fontsize':10}
    
    smooth = lowess(residual,fitted_value)
    ax[0,0].scatter(x=fitted_value,y=residual,s=10,c='#1b6ca8')
    ax[0,0].plot(smooth[:,0],smooth[:,1],color='green')
    ax[0,0].hlines(y=0,xmin=fitted_value.min(),xmax=fitted_value.max(),linestyles='--',color='black')
    ax[0,0].set_title('Linearity \nReference line should be flate and horizontal',loc='left',fontdict=fontdict)
    ax[0,0].set_xlabel('Fitted values')
    ax[0,0].set_ylabel('Residuals')

    ax[0,1].set_title('Homogeneity of Variance \nReference line should be flat and horizontal',loc='left',fontdict=fontdict)
    ax[0,1].set_xlabel('Fitted values')
    ax[0,1].set_ylabel('abs(std_residuals)')
    smooth = lowess(abs_std_residual,fitted_value)
    ax[0,1].scatter(x=fitted_value,y=abs_std_residual,s=10,c='#1b6ca8')
    ax[0,1].plot(smooth[:,0],smooth[:,1],color='green')
    
    
    ax[1,0].set_title('Normality of Residuals \nDots should fall along the line',loc='left',fontdict=fontdict)
    ax[1,0].set_xlabel('Standard Normal Distribution Quantiles')
    ax[1,0].set_ylabel('sample Quantiles')
    norm_quant,emp_quant = get_quantiles(std_residual)
    ax[1,0].plot([norm_quant.min(),norm_quant.max()],[norm_quant.min(),norm_quant.max()],c='green')
    ax[1,0].scatter(x=norm_quant,y=emp_quant,c='#1b6ca8',s=10)
    
    ax[1,1].set_title('Posterior Predictive Check \nModel-predicted lines should resemble observed data line',loc='left',fontdict=fontdict)
    ax[1,1].set_xlabel(y_name)
    ax[1,1].set_ylabel('Density')
    for pred in boot_pred:
        plot_kde(pred,ax[1,1],c='#1b6ca8',alpha=0.1)
    plot_kde(y_value,ax[1,1],c='green')
    plt.show()

def get_quantiles(x):
    x = np.sort(x)
    rank = (np.argsort(x)+1) / (len(x)+1)
    norm_quant = norm(loc=0,scale=1).ppf(rank)
    emp_quant = (x-np.mean(x))/np.std(x)
    return norm_quant,emp_quant

def plot_kde(x,ax,c,linewidth=1,alpha=1):
    x = np.sort(x)
    pdf = gaussian_kde(x).pdf(x)
    ax.plot(x,pdf,color=c,linewidth=linewidth,alpha=alpha)
    ax.ticklabel_format(style='sci',axis='y',scilimits=[0,1],useMathText=True)
    ax.get_yaxis().get_offset_text().set_visible(False)

rng = np.random.default_rng(0)
x = np.linspace(5,5.1,1000)
y = rng.normal(loc=0,scale=1,size=1000)
bp = [rng.normal(loc=1,scale=1,size=1000) for i in range(30)]

plot_check_model(residual=y,fitted_value=x,boot_pred=bp)